In [1]:

import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D,BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

from glob import glob
import os


In [2]:
emotion_to_test=[]
path_to_test = []
for dirname, _, filenames in os.walk('/kaggle/input/brain-dead-emotion-detection/brain_dead_emotion_detection/Test'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        emotion_to_test.append(path.split('/')[-2])
        path_to_test.append(path)
        
test = pd.DataFrame()
test['path']= path_to_test
test['emotion']= emotion_to_test

In [3]:
emotion_to_train=[]
path_to_train = []
for dirname, _, filenames in os.walk('/kaggle/input/brain-dead-emotion-detection/brain_dead_emotion_detection/Train'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        emotion_to_train.append(path.split('/')[-2])
        path_to_train.append(path)
train = pd.DataFrame()
train['path']= path_to_train
train['emotion']= emotion_to_train

In [20]:
class CFG:
    num_classes = 4
    width = 124
    height = 124
    channel = 3
    batch_size=2
    EPOCHS = 30
    lr = 0.00001

In [21]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet', 
                                include_top=False, 
                                input_shape=(CFG.width,CFG.height ,CFG.channel ))
base_model.trainable = False

x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)

x = Dropout(0.5)(x)

x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

predictions = Dense(CFG.num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, predictions)

# model.summary()

In [22]:
TRAIN_DATAGEN = ImageDataGenerator(horizontal_flip=True,
                                   rotation_range=25,
                                   brightness_range=[0.15,.045],
                                   rescale = 1./255,
                )

TEST_DATAGEN = ImageDataGenerator(
                rescale = 1.0/255
                )

train_flow = TRAIN_DATAGEN.flow_from_dataframe(
                                    dataframe=train,
                                    x_col="path",
                                    y_col="emotion",
                                    subset="training",
                                    batch_size=CFG.batch_size,
                                    seed=42,
                                    shuffle=True,
                                    class_mode="categorical",
                                    target_size=(CFG.width,CFG.height))

test_flow = TEST_DATAGEN.flow_from_dataframe(
                                    dataframe=test,
                                    x_col="path",
                                    y_col="emotion",
                                    subset="training",
                                    batch_size=CFG.batch_size,
                                    seed=42,
                                    shuffle=True,
                                    class_mode="categorical",
                                    target_size=(CFG.width,CFG.height))

TRAINING_NUM = train_flow.n 
VALID_NUM = test_flow.n

STEP_SIZE_TRAIN = TRAINING_NUM // CFG.batch_size 
STEP_SIZE_VALID = VALID_NUM // CFG.batch_size

model.compile(loss=tf.keras.losses.KLDivergence()
              , optimizer=tf.keras.optimizers.Adam(CFG.lr), metrics=[tf.keras.metrics.Precision(name='precision')])

reduce_lr=ReduceLROnPlateau(monitor='val_precision'
                            ,factor=0.1,patience=5, min_delta=1e-3, verbose=1, min_lr=1e-6)

weights=ModelCheckpoint('model_weights.hdf5',
                       save_best_only=True,
                       monitor='val_precision',
                       verbose=1,
                       save_weights_only=False
                       )

early_stopping=EarlyStopping(monitor='val_precision',patience=5,
                             restore_best_weights=True)


Found 1810 validated image filenames belonging to 4 classes.
Found 369 validated image filenames belonging to 4 classes.


In [23]:

history = model.fit(
                    train_flow,
                    validation_data=test_flow,
                    epochs=CFG.EPOCHS,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[reduce_lr,weights,early_stopping]
                    )

Epoch 1/30
904/905 [============================>.] - ETA: 0s - loss: 1.3735 - precision: 0.3000
Epoch 1: val_precision improved from inf to 0.46429, saving model to model_weights.hdf5
905/905 [==============================] - 22s 20ms/step - loss: 1.3735 - precision: 0.3000 - val_loss: 1.3455 - val_precision: 0.4643 - lr: 1.0000e-05
Epoch 2/30
904/905 [============================>.] - ETA: 0s - loss: 1.3238 - precision: 0.5789
Epoch 2: val_precision did not improve from 0.46429
905/905 [==============================] - 17s 19ms/step - loss: 1.3238 - precision: 0.5789 - val_loss: 1.2803 - val_precision: 0.5323 - lr: 1.0000e-05
Epoch 3/30
903/905 [============================>.] - ETA: 0s - loss: 1.2907 - precision: 0.5926
Epoch 3: val_precision did not improve from 0.46429
905/905 [==============================] - 16s 18ms/step - loss: 1.2908 - precision: 0.5926 - val_loss: 1.2487 - val_precision: 0.6119 - lr: 1.0000e-05
Epoch 4/30
902/905 [============================>.] - ETA: 0s